<a href="https://colab.research.google.com/github/TaruSora/NLP_100knock/blob/main/100knock_9_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install torch==1.6.0
! pip install torchtext==0.7.0
! pip install torch_optimizer
! pip install pytorch-lightning==1.0.8

     |████████████████████████████████| 748.8 MB 18 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
     |████████████████████████████████| 4.5 MB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
     |████████████████████████████

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip
!sed -e 's/"/'\''/g' newsCorpora.csv > newsCorpora_re.csv

--2022-03-31 10:54:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  31.9MB/s    in 0.9s    

2022-03-31 10:54:03 (31.9 MB/s) - ‘NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  NewsAggregatorDataset.zip
  inflating: 2pageSessions.csv       
   creating: __MACOSX/
  inflating: __MACOSX/._2pageSessions.csv  
  inflating: newsCorpora.csv         
  inflating: __MACOSX/._newsCorpora.csv  
  inflating: readme.txt              
  inflating: __MACOSX/._readme.txt   


In [ ]:
# 50のコードをそのまま利用

import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('newsCorpora_re.csv', header=None, sep='\t', names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
df = df.loc[df['PUBLISHER'].isin(['Reuters', 'Huffington Post', 'Businessweek', 'Contactmusic.com', 'Daily Mail']), ['TITLE', 'CATEGORY']]

train, tmp = train_test_split(df, test_size=0.2, shuffle=True, random_state=42, stratify=df['CATEGORY'])
valid, test = train_test_split(tmp, test_size=0.5, shuffle=True, random_state=42, stratify=tmp['CATEGORY'])

train.to_csv('train.txt', sep='\t', index=False)
valid.to_csv('valid.txt', sep='\t', index=False)
test.to_csv('test.txt', sep='\t', index=False)

print('学習用データ')
print(train['CATEGORY'].value_counts())
print('評価用データ')
print(valid['CATEGORY'].value_counts())
print('テスト用データ')
print(test['CATEGORY'].value_counts())

学習用データ
b    4501
e    4235
t    1220
m     728
Name: CATEGORY, dtype: int64
評価用データ
b    563
e    529
t    153
m     91
Name: CATEGORY, dtype: int64
テスト用データ
b    563
e    530
t    152
m     91
Name: CATEGORY, dtype: int64


80.問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [ ]:
# 80 ID番号への変換

import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
import torch_optimizer as optim
from torch.utils.data import DataLoader
from torchtext.data import Example, Field, Dataset, BucketIterator
import string
import os

# データ読み込み
text_field = Field(sequential=True, use_vocab=True)
label_field = Field(sequential=False, use_vocab=False, is_target=True)
fields = [("x", text_field), ("t", label_field)]

table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
label2id = {'b': 0, 't': 1, 'e': 2, 'm': 3}
batch_size = 1

def load_corpus(fname):
    examples = list()
    with open(fname, 'r') as f:
        sentences = df["TITLE"]
        labels = df["CATEGORY"]
        for sentence, label in zip(sentences, labels):
            word_list = sentence.translate(table).split()
            label_id = label2id[label]
            examples.append(Example.fromlist([word_list, label_id], fields))
        return Dataset(examples, fields)

train_dataset = load_corpus('train.txt')
valid_dataset = load_corpus('valid.txt')
test_dataset = load_corpus('test.txt')

# 訓練データにある単語のうち2回以上出現した単語にIDを振り分ける
text_field.build_vocab(train_dataset, min_freq=2)

train_dataloader = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = BucketIterator(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = BucketIterator(test_dataset, batch_size=batch_size, shuffle=False)

text = train.iloc[1, train.columns.get_loc('TITLE')]
def id(sentence, text_field=text_field):
    return [ text_field.vocab.stoi[word] for word in sentence.translate(table).split()]
print(f'テキスト：{text}')
print(f'ID列：{id(text)}')

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)


テキスト：Iraq concerns lift top-rated euro zone bonds but Fed limits gains
ID列：[193, 571, 1380, 314, 4180, 93, 284, 624, 99, 42, 3618, 240]


/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


81.RNN

In [ ]:
# 81 RNNによる予測

class MyRNN(pl.LightningModule):
    def __init__(self, n_input, n_emb, n_hidden, n_layers, n_output, dropout, bidirectional):
        super(MyRNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_emb, padding_idx=1)
        self.lstm = nn.LSTM(input_size=n_emb, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)

    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[:, -1, :])

n_input = len(text_field.vocab)
n_emb = 300
n_hidden = 50
n_layers = 1
n_output = len(label2id)
dropout = 0.1
bidirectional = False

model = MyRNN(n_input, n_emb, n_hidden, n_layers, n_output, dropout, bidirectional)

for i, data in enumerate(train_dataloader):
    if i>9:
        break
    result = torch.tensor([data.x[i].item() for i in range(len(data.x))])
    print(torch.softmax(model(result.unsqueeze(0)), dim=-1))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[0.2602, 0.2345, 0.2128, 0.2924]], grad_fn=<SoftmaxBackward>)
tensor([[0.2528, 0.2532, 0.2595, 0.2345]], grad_fn=<SoftmaxBackward>)
tensor([[0.3129, 0.1951, 0.2003, 0.2917]], grad_fn=<SoftmaxBackward>)
tensor([[0.2971, 0.2567, 0.2046, 0.2416]], grad_fn=<SoftmaxBackward>)
tensor([[0.3411, 0.1993, 0.2229, 0.2367]], grad_fn=<SoftmaxBackward>)
tensor([[0.2680, 0.2573, 0.2317, 0.2430]], grad_fn=<SoftmaxBackward>)
tensor([[0.2525, 0.2797, 0.2559, 0.2119]], grad_fn=<SoftmaxBackward>)
tensor([[0.2802, 0.1936, 0.2466, 0.2795]], grad_fn=<SoftmaxBackward>)
tensor([[0.2688, 0.2555, 0.2349, 0.2408]], grad_fn=<SoftmaxBackward>)
tensor([[0.2727, 0.2295, 0.2671, 0.2307]], grad_fn=<SoftmaxBackward>)


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


82.確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

In [ ]:
# 82 確率的勾配降下法による学習

class MyRNN(pl.LightningModule):
    # 層の定義
    def __init__(self, n_input, n_emb, n_hidden, n_layers, n_output, dropout, bidirectional):
        super(MyRNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_emb, padding_idx=1)
        self.lstm = nn.LSTM(input_size=n_emb, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)    
    # 順伝播
    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[-1])
    # 訓練用データの損失計算
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("train_loss", loss)
        return loss
    # 検証用データの損失計算
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("val_loss", loss)
    # 評価用データの正解率計算
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x) 
        y = torch.argmax(y, dim=1)
        accuracy = torch.sum(t == y).item() / (len(y) * 1.0)
        self.log("test_acc", accuracy)
    # 損失関数の定義
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    # オプティマイザの定義
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1) 

model2 = MyRNN(n_input, n_emb, n_hidden, n_layers, n_output, dropout, bidirectional)

# 検証用データの損失が最小のモデルを保存
checkpoint = pl.callbacks.ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, save_weights_only=True, dirpath="model/")

early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=10, callbacks=[checkpoint, early_stopping])
trainer.fit(model2, train_dataloader, valid_dataloader)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=test_dataloader)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | embed | Embedding | 3.2 M 
1 | lstm  | LSTM      | 70.4 K
2 | fc    | Linear    | 204   


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=7.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.1262, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9555),
 'train_loss': tensor(0.0023, device='cuda:0'),
 'val_loss': tensor(0.1528, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.956


83.問題82のコードを改変し，B事例ごとに損失・勾配を計算して学習を行えるようにせよ（Bの値は適当に選べ）．また，GPU上で学習を実行せよ．

In [ ]:
# 83 ミニバッチ化・GPU上での学習

batch_size = 32

train_dataloader = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = BucketIterator(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = BucketIterator(test_dataset, batch_size=batch_size, shuffle=False)

n_input = len(text_field.vocab)
n_embed = 300
n_hidden = 50
n_layers = 1
n_output = len(label2id)
dropout = 0.3
bidirectional = False

model = MyRNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional)

# 検証用データの損失が最小のモデルを保存
checkpoint = pl.callbacks.ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, save_weights_only=True, dirpath="model/")

early_stopping = pl.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5)

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=10, callbacks=[checkpoint, early_stopping])
trainer.fit(model2, train_dataloader, valid_dataloader)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=test_dataloader)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory model/ exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
/usr/local/lib/p

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=9.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.0982, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9711),
 'train_loss': tensor(0.2147, device='cuda:0'),
 'val_loss': tensor(0.0982, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.971


84.事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル）で単語埋め込みemb(x)を初期化し，学習せよ．

In [ ]:
# 84 単語ベクトルの導入

# 単語ベクトルの読み込み
from torchtext.vocab import FastText
text_field.build_vocab(train_dataset, vectors=FastText(language="en"), min_freq=2)

ストリーミング出力は最後の 5000 行に切り捨てられました。
100%|██████████| 2519370/2519370 [10:00<00:00, 4194.11it/s]


In [ ]:
class MyRNN(pl.LightningModule):
    # 層の定義
    def __init__(self, n_input, n_emb, n_hidden, n_layers, n_output, dropout, bidirectional, lr):
        super(MyRNN, self).__init__()
        self.embed = nn.Embedding(num_embeddings=n_input, embedding_dim=n_emb, padding_idx=1)
        self.embed.weight.data.copy_(text_field.vocab.vectors)
        self.lstm = nn.LSTM(input_size=n_embed, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.lstm = nn.LSTM(input_size=n_emb, hidden_size=n_hidden, num_layers=n_layers, dropout=dropout, bidirectional=bidirectional)
        self.fc = nn.Linear(in_features=n_hidden * (2 if bidirectional==True else 1), out_features=n_output)    
    # 順伝播
    def forward(self, x):
        o, (h, c) = self.lstm(self.embed(x))
        return self.fc(o[-1])
    # 訓練用データの損失計算
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("train_loss", loss)
        return loss
    # 検証用データの損失計算
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.lossfun(y, t)
        self.log("val_loss", loss)
    # 評価用データの正解率計算
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x) 
        y = torch.argmax(y, dim=1)
        accuracy = torch.sum(t == y).item() / (len(y) * 1.0)
        self.log("test_acc", accuracy)
    # 損失関数の定義
    def lossfun(self, y, t):
        return F.cross_entropy(y, t)
    # オプティマイザの定義
    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.1)

In [ ]:
batch_size = 32

train_dataloader = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = BucketIterator(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = BucketIterator(test_dataset, batch_size=batch_size, shuffle=False)

n_input = len(text_field.vocab)
n_embed = 300
n_hidden = 50
n_layers = 1
n_output = len(label2id)
dropout = 0.1
bidirectional = False
lr = 0.1

model3 = MyRNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

checkpoint = pl.callbacks.ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1,save_weights_only=True, dirpath="model/")

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=10, callbacks=[checkpoint])
trainer.fit(model, train_dataloader, valid_dataloader)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=test_dataloader)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory model/ exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
/usr/local/lib/p

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=9-v0.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.2023, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9350),
 'train_loss': tensor(0.2516, device='cuda:0'),
 'val_loss': tensor(0.2023, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.935


85.順方向と逆方向のRNNの両方を用いて入力テキストをエンコードし，モデルを学習せよ．
さらに，双方向RNNを多層化して実験せよ．

In [ ]:
# 85 双方向RNN・多層化

batch_size = 32

train_dataloader = BucketIterator(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = BucketIterator(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = BucketIterator(test_dataset, batch_size=batch_size, shuffle=False)

n_input = len(text_field.vocab)
n_embed = 300
n_hidden = 50
n_layers = 3
n_output = len(label2id)
dropout = 0.1
bidirectional = True
lr = 0.1

model4 = MyRNN(n_input, n_embed, n_hidden, n_layers, n_output, dropout, bidirectional, lr)

checkpoint = pl.callbacks.ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, save_weights_only=True, dirpath="model/")

# 訓練
trainer = pl.Trainer(gpus=1, max_epochs=10, callbacks=[checkpoint])
trainer.fit(model4, train_dataloader, valid_dataloader)

# ベストモデルの確認
print("ベストモデル: ", checkpoint.best_model_path)
print("ベストモデルの検証用データにおける損失: ", checkpoint.best_model_score)

# 評価
test = trainer.test(test_dataloaders=test_dataloader)
print("Test accuracy = %.3f" % (test[0]["test_acc"]))

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory model/ exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

ベストモデル:  /content/model/epoch=9-v1.ckpt
ベストモデルの検証用データにおける損失:  tensor(0.3739, device='cuda:0')


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.8491),
 'train_loss': tensor(0.3935, device='cuda:0'),
 'val_loss': tensor(0.3739, device='cuda:0')}
--------------------------------------------------------------------------------
Test accuracy = 0.849
